## Deliverable 3. Create a Travel Itinerary Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import sys
sys.path.append("..")
# Import API key
from config import g_key

import os
# Configure gmaps
gmaps.configure(api_key=g_key)

In [34]:
gmaps.__version__

'0.9.0'

In [65]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv(os.path.join("..", "Vacation_Search", "WeatherPy_vacation.csv"))
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Iquique,CL,75.24,clear sky,-20.2208,-70.1431,Backpacker`s Hostel Iquique
1,4,Labuhan,ID,78.13,overcast clouds,-6.8844,112.2051,DoubleFAG
2,5,Ndungu,TZ,72.43,light rain,-4.3667,38.0500,Nick Resort
3,7,Meulaboh,ID,77.63,overcast clouds,4.1363,96.1285,Kost Pria
4,12,Kokopo,PG,76.01,overcast clouds,-4.3520,152.2633,Kokopo Beach Bungalow Resort


In [36]:
place = vacation_df.drop_duplicates(subset = ["Lat", "Lng"])
place.loc[place["Country"] == "BR"]

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
15,39,Caravelas,BR,80.96,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
24,55,Botucatu,BR,77.72,few clouds,-22.8858,-48.4450,Primar Plaza Hotel
29,73,Castro,BR,77.47,scattered clouds,-24.7911,-50.0119,CHACARA BAILLY
35,87,Juruti,BR,82.31,overcast clouds,-2.1522,-56.0922,Pousada Marupiara
49,117,Carauari,BR,84.74,overcast clouds,-4.8828,-66.8958,Hotel Fellip
74,181,Itarema,BR,82.18,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
80,202,Maceio,BR,85.44,clear sky,-9.6658,-35.7353,Poutur Pousada
104,251,Mazagao,BR,89.65,scattered clouds,-0.1150,-51.2894,Recanto dos Ferreiras
105,252,Touros,BR,86.90,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
122,291,Pitimbu,BR,86.59,few clouds,-7.4706,-34.8086,Reserva do Abiaí


In [37]:
place["Country"].value_counts()

BR    26
US    24
AU    15
ID    13
PE    12
      ..
VU     1
DO     1
BO     1
YE     1
CV     1
Name: Country, Length: 72, dtype: int64

In [64]:
vacation_df.loc[vacation_df["Country"] == "US"].head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,16,Hilo,US,77.11,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
7,17,Fortuna,US,66.92,clear sky,40.5982,-124.1573,Super 8 by Wyndham Fortuna
12,22,Sterling,US,60.35,clear sky,39.0062,-77.4286,Hampton Inn & Suites Washington-Dulles Interna...
18,45,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
26,60,Charleston,US,70.21,scattered clouds,32.7766,-79.9309,The Mills House Wyndham Grand Hotel


In [39]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [40]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [56]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
start_city = "Florida"
end_city = "Florida"
city1 = "Newton"
city2 = "Eureka"
city3 = "Rincon"

vacation_start = vacation_df.loc[vacation_df["City"] == start_city]
vacation_end = vacation_df.loc[vacation_df["City"] == end_city ]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == city1]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == city2] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == city3] 

In [57]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start['Lat'].to_numpy()[0], vacation_start['Lng'].to_numpy()[0]
end = vacation_end['Lat'].to_numpy()[0], vacation_end['Lng'].to_numpy()[0]
stop1 = vacation_stop1['Lat'].to_numpy()[0], vacation_stop1['Lng'].to_numpy()[0]
stop2 = vacation_stop2['Lat'].to_numpy()[0], vacation_stop2['Lng'].to_numpy()[0]
stop3 = vacation_stop3['Lat'].to_numpy()[0], vacation_stop3['Lng'].to_numpy()[0]

In [58]:
#start = (42.65, -73.75)
#end = (42.65, -73.75)
#stop1 =(37.46, -122.43)
#stop2 = (37.22, -93.29)
#stop3 = (40.8, -124.16)


In [59]:
print(start, end, stop1, stop2, stop3)

(28.7505, -82.5001) (28.7505, -82.5001) (42.337, -71.2092) (40.8021, -124.1637) (32.296, -81.2354)


In [60]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig2 = gmaps.figure()
travel_route = gmaps.directions_layer(start, end, waypoints =[stop1, stop2, stop3], travel_mode="DRIVING")
fig2.add_layer(travel_route)
fig2

Figure(layout=FigureLayout(height='420px'))

In [61]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,457,Florida,US,83.21,clear sky,28.7505,-82.5001,Sportsmens Cove
1,467,Newton,US,61.84,clear sky,42.3370,-71.2092,Hotel Indigo Boston - Newton Riverside
2,280,Eureka,US,67.19,clear sky,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
3,560,Rincon,US,68.45,overcast clouds,32.2960,-81.2354,Days Inn Rincon


In [62]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [63]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))